<p style="text-align:right"><img  src="https://postmba.org/www/wp-content/uploads/2021/10/PostMBA-logo.png" width="150" alt="regression"></p>


<p style="text-align:center"><img  src="https://www.investopedia.com/thmb/EuoiCsbidRDdGqTnLps5ycTsNVY=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/BETA-FINAL-9a06ac0fc2d84589aded1475150869c2.jpg" width="500" alt="regression"></p>


<p dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium;align:left">
<font face="Arial" size=3><b>
Created by :
Hosein Ahmadi</b>
</font>
</p>


### import packages

In [21]:
import finpy_tse as fpy
import pandas as pd
import statsmodels.api as sm
import yfinance as yf
import plotly.express as px

### get data

In [22]:
def get_usa_data(symbol):
    data = yf.Ticker(symbol).history(period="60mo")
    data.rename({'Close': 'Adj Close'}, axis=1, inplace=True)
    data['Ticker'] = symbol
    return data.resample('M').last()[['Ticker', 'Adj Close']]

def get_iran_data(symbol):
    data = fpy.Get_Price_History(
    stock=symbol,
    start_date='1397-08-01',
    end_date='1402-07-01',
    ignore_date=False,
    adjust_price=True,
    show_weekday=False,
    double_date=True)[['Date', 'Ticker', 'Adj Close']]
    
    data = data.set_index('Date')
    
    return data.resample('M').last()

def get_monthly_tepix():
    data = fpy.Get_CWI_History(
        start_date='1397-07-01',
        end_date='1402-07-01',
        ignore_date=False,
        just_adj_close=False,
        show_weekday=False,
        double_date=True)
    
    data = data.set_index('Date')
    data['Ticker'] = 'tepix'
    return data.resample('M').last()[['Ticker', 'Adj Close']]

In [23]:
symbol = 'غگل'

In [ ]:
stock_data = get_iran_data(symbol)
tepix_data = get_monthly_tepix()

In [189]:
data = stock_data.merge(tepix_data, left_index=True, right_index=True, suffixes = ['_stock', '_tepix'])
monthly_return = data[['Adj Close_stock', 'Adj Close_tepix']].pct_change().dropna()
monthly_return.head()

,Adj Close_stock,Adj Close_tepix
Date,,
2018-11-30,-0.126316,-0.106863
2018-12-31,0.126506,-0.014452
2019-01-31,0.160428,-0.006880
2019-02-28,-0.069124,0.031194
2019-03-31,0.123762,0.120911


In [190]:
risk_free_return = 0.2
extra_monthly_return = monthly_return - risk_free_return
extra_monthly_return.head()

,Adj Close_stock,Adj Close_tepix
Date,,
2018-11-30,-0.326316,-0.306863
2018-12-31,-0.073494,-0.214452
2019-01-31,-0.039572,-0.206880
2019-02-28,-0.269124,-0.168806
2019-03-31,-0.076238,-0.079089


### calculate beta 

In [215]:
def get_Beta(ticker, is_usa = False):
    
    #get data and preparation
    if is_usa:
        stock_data = get_usa_data(ticker)
        sp500_data = get_usa_data('^GSPC')
        data = stock_data.merge(sp500_data, left_index=True, right_index=True, suffixes = ['_stock', '_index'])
        
    else:
        stock_data = get_iran_data(ticker)
        tepix_data = get_monthly_tepix()
        data = stock_data.merge(tepix_data, left_index=True, right_index=True, suffixes = ['_stock', '_index'])
    
    monthly_return = data[['Adj Close_stock', 'Adj Close_index']].pct_change().dropna()
    extra_monthly_return = monthly_return - risk_free_return
    
    #calculate beta
    data = extra_monthly_return
    X = sm.add_constant(data['Adj Close_index'])
    y = data['Adj Close_stock']
    est = sm.OLS(y, X).fit()
    params = (est.params)
    tvalues = (est.tvalues)
    pvalues = est.pvalues
    
    #show outputs
    results = pd.concat([params, tvalues], axis=1)
    results['p-values'] = pvalues
    results.rename({0: 'coeff', 1: 't-stats'}, axis=1, inplace=True)
    results.index = ['_alpha', '_Beta']
    results['stock'] = ticker

    return (results.round(2))

In [216]:
get_Beta(ticker = 'شتران')

,coeff,t-stats,p-values,stock
_alpha,0.08,2.84,0.01,شتران
_Beta,1.44,10.39,0.00,شتران


In [218]:
get_Beta(ticker = 'MSFT', is_usa = True)

,coeff,t-stats,p-values,stock
_alpha,-0.01,-0.32,0.75,MSFT
_Beta,0.90,8.97,0.00,MSFT


## more analysis

In [254]:
persian_symbols = ['ثاخت', 'ثشاهد', 'سصفها', 'خپارس', 'شتران', 'وبصادر', 'شستا', 'وتجارت', 'فملی', 'فارس']

In [255]:
results = pd.DataFrame()
for symbol in persian_symbols:
    output = get_Beta(symbol).drop('_alpha', axis=0)
    results = pd.concat([results, output], axis=0)

In [256]:
results = results.set_index('stock').rename({'coeff': 'Beta'}, axis=1)
results

,Beta,t-stats,p-values
stock,,,
ثاخت,1.06,9.28,0.0
ثشاهد,1.79,9.81,0.0
سصفها,0.92,3.20,0.0
خپارس,1.39,13.10,0.0
شتران,1.44,10.39,0.0
وبصادر,1.11,10.62,0.0
شستا,1.30,7.60,0.0
وتجارت,1.02,8.79,0.0
فملی,1.32,14.01,0.0


In [257]:
 px.bar(results.sort_values('Beta'), y=["Beta"])

In [258]:
persian_symbols = ['AAPL', 'NVDA', 'TSLA', 'META', 'KO', 'AMZN', 'NFLX']
usa_results = pd.DataFrame()
for symbol in persian_symbols:
    output = get_Beta(symbol, is_usa = True).drop('_alpha', axis=0)
    usa_results = pd.concat([usa_results, output], axis=0)

In [259]:
usa_results = usa_results.set_index('stock').rename({'coeff': 'Beta'}, axis=1)
usa_results

,Beta,t-stats,p-values
stock,,,
AAPL,1.31,9.16,0.0
NVDA,1.68,5.81,0.0
TSLA,2.24,5.04,0.0
META,1.21,4.93,0.0
KO,0.59,5.56,0.0
AMZN,1.19,6.77,0.0
NFLX,1.26,5.15,0.0


In [260]:
px.bar(usa_results.sort_values('Beta'), y=["Beta"])

<p dir=rtl style="direction: rtl;text-align: right;line-height:175%;font-family:vazir;font-size:medium;align:left">
    <font face="B nazanin" size=3>
        <b>
            نتیجه‌گیری:
        </b>
        <br>
            به منظور انجام آزمون فرض بتا، میتونیم از آمار تی-استیودنت استفاده کنیم زیرا که صرفا یک متغیر مورد آزمون قرار می گیرد. فرض صفر در این آزمون صفر بودن بتا و فرض مخالف بتای متفاوت با صفر است. همانطور نتایج برای همه سهم‌ها نشان می‌دهد، مقدار آماره آزمون به قدری بزرگ است که میزان p-value به قدری کوچک شده است که احتمال رد فرض صفر بسیار کم است و در سطح ۱٪ همگی رد می‌شوند.
          
    
</p>